<a href="https://colab.research.google.com/github/Hee-won/Hee-won.github.io/blob/master/Classifying_passport_pictures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**여권사진 분류하기 프로젝트 전체 코드 Classifying passport pictures**

1.   이중분류
2.   다중분류

이중분류 Two-way classification


In [ ]:
# 1. 구글 드라이브 연동

from google.colab import drive
drive mount('/content/drive')

In [ ]:
# 2. zip 파일 압축 풀기

import zipfile
with zipfile.Zipfile('content/drive/My Drive/data_set.zip', 'r')as zip_ref:
  zip_ref.extractall('/content/drive/My Drive/data_set')

In [ ]:
# 3. Data set 불러오기
from tensorflow import keras

# 훈련셋 + data exaggeration
train_datagen = keras.preprocessing.image.ImaeDataGenerator(rescale = 1/255, horizontal_flip = True)
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/data_set/train', target_size = (413, 531), batch_size = 10, class_mode = 'binary', shuffle = True)

# 평가셋
Test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
Test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/data_set/test', target_size = (413, 531), batch_size = 10, class_mode = 'binary')
print(train_generator.image_shape)
print(train_generator.class_indices)

In [ ]:
# 4. data 살펴보기
print(test_generator[0][0].shape)

In [ ]:
# 5. 모델 구성

from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
model=keras.Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu',input_shape = test_generator.image_shape))
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
# 6. compile
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# 7. train
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience = 15)
epochs = 100
history = model.fit(train_generator,steps_per_epoch = 100, epochs = epochs, validation_data = test_generator, validation_steps = 20, callbacks = [early_stop])

In [ ]:
# 8. Evaluate
result = model.evaluate(test_generator, steps = 1)
print(result)

In [ ]:
# 9. 시각화
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
# 10. 모델 저장
model_path1 = 'passport_wrongornot.h5'
model.save(model_path1)

다중분류 Multicategory classification

In [ ]:
# 11. zip 파일 압축 풀기
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/data_set2.zip', 'r') as zip_ref:
zip_ref.extractall('/content/drive/My Drive/data_set2')

In [ ]:
# 12. 데이터셋 불러오기
from tensorflow import keras
# 훈련셋 + data exaggeration
train_datagen2 = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255, horizontal_flip = True)
train_generator2 = train_datagen2.flow_from_directory('/content/drive/My Drive/data_set2/train', target_size = (413, 531), batch_size = 20, class_mode = 'categorical', shuffle = True)
# 평가셋
Test_datagen2 = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
Test_generator2 = test_datagen2.flow_from_directory('/content/drive/My Drive/data_set2/test', target_size = (413, 531), batch_size = 20, class_mode = 'categorical')
print(train_generator2.image_shape)
print(train_generator2.class_indices)

In [ ]:
# 13. data 살펴보기
print(test_generator2[0][0].shape)

In [ ]:
# 14. 모델 구성
model2 = keras.Sequential()
model2.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = test_generator.image_shape, padding = 'same'))
model2.add(Conv2D(32,(3,3),activation = 'relu', padding = 'same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(128, (3,3),activation = 'relu', padding = 'same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Flatten())
model2.add(Dense(64, activation = 'relu'))
model2.add(Dense(32, activation = 'relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5, activation = 'softmax'))
model2.summary()

In [ ]:
# 15. compile
model2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# 16. train
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience = 15)
epochs = 100
history2 = model2.fit(train_generator2, steps_per_epoch = 50, epochs = epochs, validation_data = test_generator2, validation_steps = 10, callbacks = [early_stop])

In [ ]:
# 17. evaluate
result2 = model2.evaluate(test_generator2, steps = 1)
print(result2)

In [ ]:
# 18. 시각화

import matplotlib.pyplot as plt
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
# 19. 모델 저장
model_path2 = 'passport_wrongtype.h5'
model2.save(model_path2)

In [ ]:
# 20. 모델 불러오기
from keras.models import load_model
model_1 = load_model(model_path1)
model_2 = load_model(model_path2)
print(model_1.summary())
print(model_2.summary())

In [ ]:
# 21. 새로운 이미지 테스트
from google.colab import files
upload = files.upload()

In [ ]:
# 22. 이미지 확인
import cv2
test_image = cv2.imread('image.png')
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
plt.imshow(test_image)

In [ ]:
# 23. resize -> (413, 531, 3)
image_resized = cv2.resize(test_image, dsize=(531,413), interpolation=cv2.INTER_AREA)
print(image_resized.shape)

In [ ]:
# 24. test
import numpy as np
result = model_1.predict(image_resized.reshape(1, 413, 531, 3))
if result == 0:
  result2 = model_2.predict(image_resized.reshape(1, 413, 531, 3))
  score = result2[0]
  print("abnormal:")
  print(print(np.where(result2 == np.amax(result2))[1]))
else:
  print("normal: ")
  print(print(np.where(result == np.amax(result))[1]))